In [9]:
#To increase cell width of ipynb
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
import tensorflow as tf
from collections import defaultdict
import gzip
import cv2 as cv
import os
import numpy as np
import random

/home/sudhakaran/Desktop/3d_recognition_thesis/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sudhakaran/Desktop/3d_recognition_thesis/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sudhakaran/Desktop/3d_recognition_thesis/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)]

In [4]:
class DEN():

    def __init__(self):
        self.last_label_index = 0    # tracks the label index upto which the model has been trained
        self.params = dict()
        self.k_ex = 5
        self.den_layers = 3
        self.conv_layers = 2
        tf.reset_default_graph()
        self.sess = None
        self.train = []
        self.train_labels = []
        self.test = []
        self.test_labels = []
        self.batch_size = 10
        self.epochs = 50
        self.early_stop = 5
        self.selected = dict()
        self.lr = 1e-3
        self.l2_mu = 0.001
        self.lamba_regular = 0.5
        self.l1_thr = 0.00001
#         self.l1_thr = 0.00001
        self.loss_thr = 0.01

    def extract_data(self):
        train = []
        test = []
        train_labels = []
        test_labels = []
        label_names = []
        
        # For easier disk read operation
#         VGG16
#         np_train_path = '../ModelNet40/extracted_features/VGG16/train.npy.gz'
#         np_test_path = '../ModelNet40/extracted_features/VGG16/test.npy.gz'
#         np_train_label_path = '../ModelNet40/extracted_features/VGG16/train_labels.npy'
#         np_test_label_path = '../ModelNet40/extracted_features/VGG16/test_labels.npy'
#         np_label_names_path = '../ModelNet40/extracted_features/VGG16/label_names.npy'

#       MobileNetV2
        np_train_path = '../ModelNet40/extracted_features/MobileNetV2/train.npy.gz'
        np_test_path = '../ModelNet40/extracted_features/MobileNetV2/test.npy.gz'
        np_train_label_path = '../ModelNet40/extracted_features/MobileNetV2/train_labels.npy'
        np_test_label_path = '../ModelNet40/extracted_features/MobileNetV2/test_labels.npy'
        np_label_names_path = '../ModelNet40/extracted_features/MobileNetV2/label_names.npy'
        
#       Custom MobileNetV2
#         np_train_path = '../ModelNet40/extracted_features/custom_MobileNetV2/train.npy.gz'
#         np_test_path = '../ModelNet40/extracted_features/custom_MobileNetV2/test.npy.gz'
#         np_train_label_path = '../ModelNet40/extracted_features/custom_MobileNetV2/train_labels.npy'
#         np_test_label_path = '../ModelNet40/extracted_features/custom_MobileNetV2/test_labels.npy'
#         np_label_names_path = '../ModelNet40/extracted_features/custom_MobileNetV2/label_names.npy'
        
        print("\n..........loading dataset from numpy files..........\n")

        with gzip.GzipFile(np_train_path, "r") as f:
            train = np.load(f)
        with gzip.GzipFile(np_test_path, "r") as f:
            test = np.load(f)

        train_labels = np.load(np_train_label_path)
        test_labels = np.load(np_test_label_path)
        label_names = np.load(np_label_names_path)

        return train, train_labels, test, test_labels, label_names


    def add_task(self, task_id, label_names, initial_output=2):

        new_label_indices = []
        self.train = []
        self.train_labels = []
        self.train_ex_labels = []      
 
        if task_id == 1:
            self.new_train = []
            self.new_train_labels = []
            self.new_train_ex_labels = []
            self.total = []
            self.total_ex_labels = []
            self.test = []
            self.test_labels = []
            for i in range(initial_output):   # By default, first task is a binary classification
                new_label_indices.append(i)
            self.last_label_index = i

        else:
            new_label_indices.append(self.last_label_index+1)
            self.last_label_index = self.last_label_index + 1
            
            #saving the old training data
            self.total = self.total + self.new_train
            self.total_ex_labels = self.total_ex_labels + self.new_train_ex_labels
            self.new_train = []
            self.new_train_labels = []
            self.new_train_ex_labels = []

        for index in new_label_indices:
            print(" \n Added new category: "+str(label_names[index]))
            l = 1 if task_id != 1 else index
            for data, label in zip(train_data, train_labels):
                if label_names[index] == label:
                    self.new_train.append(data)
                    self.new_train_labels.append(l)
                    self.new_train_ex_labels.append(index)
#                 if len(self.new_train_labels) >= 300:
#                     break

            for data, label in zip(test_data, test_labels):
                if label_names[index] == label:
                    self.test.append(data)
                    self.test_labels.append(index)
                    
        # ------------------ Random sampling old training data ------------------
#         if task_id != 1:
#             sampled_indices = random.sample(range(len(self.total)), len(self.new_train))
#             for k in sampled_indices:
#                 self.train.append(self.total[k])
#                 self.train_labels.append(0)
#                 self.train_ex_labels.append(self.total_ex_labels[k])
                
#             self.train = self.train + self.new_train
#             self.train_labels = self.train_labels + self.new_train_labels
#             self.train_ex_labels = self.train_ex_labels + self.new_train_ex_labels
#         else:
#             self.train = self.new_train
#             self.train_labels = self.new_train_labels
#             self.train_ex_labels = self.new_train_ex_labels
        
        # ------------------- Memory Replay of old training data ----------------------
        if task_id != 1:
            sample_per_label = int(len(self.new_train) / self.last_label_index)
            if sample_per_label < 20:
                sample_per_label = 20
                
            for index in range(self.last_label_index):
                label_indices = []
                for i, m in enumerate(self.total_ex_labels):
                    if m == index:
                        label_indices.append(i)
                try:
                    sampled_label_indices = random.sample(label_indices, k=sample_per_label)
                except:
                    print("\n Less samples; so taking them all")
                    sampled_label_indices = label_indices
                for k in sampled_label_indices:
                    self.train.append(self.total[k])
                    self.train_labels.append(0)
                    self.train_ex_labels.append(self.total_ex_labels[k])
                    
            self.train = self.train + self.new_train
            self.train_labels = self.train_labels + self.new_train_labels
            self.train_ex_labels = self.train_ex_labels + self.new_train_ex_labels

        else:
            self.train = self.new_train
            self.train_labels = self.new_train_labels
            self.train_ex_labels = self.new_train_ex_labels
        
        print("Train size: %d" %len(self.train_ex_labels))
        print("Test size: %d" %len(self.test_labels))
            

    def destroy_graph(self):
        tf.reset_default_graph()
        self.params = dict()

    def initialize_parameters(self, output_len=2):
        self.sess = tf.Session()
#       MobileNetV2 :-
        self.x = tf.placeholder(tf.float32, [None, 1280])
#       VGG16 :-
#         self.x = tf.placeholder(tf.float32, [None, 4 * 4 * 512])
        self.y_ = tf.placeholder(tf.float32, [None, output_len])
        # self.keep_prob = tf.placeholder(tf.float32)          # dropout probability

    def create_variable(self, name=None, shape=None, scope=None, trainable=True):
        with tf.variable_scope(scope, reuse=False):
            w = tf.get_variable(name, shape=shape, 
#                                 initializer=tf.random_normal_initializer(mean=0, stddev=1, seed=3),
                                trainable=trainable)
            if "ex" not in name:
                self.params[w.name] = w
        return w

    def get_variable(self, name=None, scope=None):
        with tf.variable_scope(scope, reuse=True):
            w = tf.get_variable(name)
            if "ex" not in name:
                self.params[w.name] = w
        return w

    def restore_params(self, task_id=None, trainable=True, param_values=dict()):
        self.params = dict()
        self.prev_W = dict()
        for scope_name, value in param_values.items():
            self.prev_W[scope_name] = value
            scope_name = scope_name.split(':')[0]
            [scope, name] = scope_name.split('/')

#             if task_id != None:
#                 if ('l%d/w_%d' % (self.den_layers,task_id) in scope_name) or ('l%d/b_%d' % (self.den_layers,task_id) in scope_name):
#                     trainable = True
#                 else:
#                     trainable = False

            with tf.variable_scope(scope, reuse=False):
                w = tf.get_variable(name, initializer=value, trainable=trainable)
            self.params[w.name] = w

    def get_params(self):
        vdict = dict()
        for scope_name, ref_w in self.params.items():
            vdict[scope_name] = self.sess.run(ref_w)
        return vdict

    def conv2d(self, x, W):
        return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

    def max_pool_2x2(self, x):
        return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                            strides=[1, 2, 2, 1], padding='SAME')


    def build_model(self, task_id, expansion=False, output_len=2):

        # Note: scope and name values are only given to DEN layers, not for fixed sized layers.

        self.initialize_parameters(output_len)

        if task_id == 1:

            #flattened first fc layer
            W_fc1 = self.create_variable(name="w", shape=[1280, 1024], scope="l1")   # (MobileNetV2) layer-1 outgoing weight matrix
#             W_fc1 = self.create_variable(name="w", shape=[4 * 4 * 512, 1024], scope="l1")   # (VGG16) layer-1 outgoing weight matrix
            b_fc1 = self.create_variable(name="b", shape=[1024], scope="l1")

            h_fc1 = tf.nn.relu(tf.matmul(self.x, W_fc1) + b_fc1)

            #second fc layer
            W_fc2 = self.create_variable(name="w", shape=[1024, 256], scope="l2")   # layer-2 outgoing weight matrix
            b_fc2 = self.create_variable(name="b", shape=[256], scope="l2")

            self.h_fc2 = tf.nn.relu(tf.matmul(h_fc1, W_fc2) + b_fc2)

            # readout fc layer
            self.w_fc = self.create_variable(name="w", shape=[256, output_len], scope="l3")     # layer-3 outgoing weight matrix 
            self.b_fc = self.create_variable(name="b", shape=[output_len], scope="l3")
            y_conv = tf.matmul(self.h_fc2, self.w_fc) + self.b_fc
            
        elif expansion:
            
            # fc layer expansion
            for layer in range(1, self.den_layers+1):

                if layer == 1:
                    w_fc1 = self.get_variable(name="w", scope="l%d"%layer)
                    b_fc1 = self.get_variable(name="b", scope="l%d"%layer)

                    w_expand = self.create_variable(name="w_ex_"+str(task_id),shape=[w_fc1.get_shape().as_list()[0], self.k_ex], scope="l"+str(layer))
                    b_expand = self.create_variable(name="b_ex_"+str(task_id),shape=[self.k_ex], scope="l%d"%layer)
                    w_expanded = tf.concat([w_fc1,w_expand],1)
                    b_expanded = tf.concat([b_fc1,b_expand],0)
                    self.params[w_fc1.name] = w_expanded
                    self.params[b_fc1.name] = b_expanded
                    h_fc1 = tf.nn.relu(tf.matmul(self.x, w_expanded) + b_expanded)

                elif layer == self.den_layers:
                    #weight matrix of current task output
                    w_fc3 = self.get_variable(name="w",scope="l%d"%layer)
                    b_fc3 = self.get_variable(name="b",scope="l%d"%layer)
                    next_dim = w_fc3.get_shape().as_list()[1]
                    
                    w_expand = self.create_variable(name="w_ex_"+str(task_id), shape=[self.k_ex, next_dim], scope="l%d"%layer)
                    w_expanded = tf.concat([w_fc3, w_expand], 0)
                    
                    self.params[w_fc3.name] = w_expanded
                    
                    y_conv = tf.matmul(self.h_fc2, w_expanded) + b_fc3

                else:
                    w_fc2 = self.get_variable(name="w",scope="l%d"%layer)
                    b_fc2 = self.get_variable(name="b",scope="l%d"%layer)

                    prev_dim = w_fc2.get_shape().as_list()[0]
                    next_dim = w_fc2.get_shape().as_list()[1]
                    
                    # Dummy nodes for prev hidden nodes
                    dummy_w = tf.get_variable(name="dummy_t%d_l%d" %(task_id,layer), shape=[self.k_ex, next_dim], 
                                initializer=tf.constant_initializer(0.0), trainable=False)

                    w_expand = self.create_variable(name="w_ex_"+str(task_id), shape=[prev_dim + self.k_ex, self.k_ex], scope="l%d"%layer)
                    b_expand = self.create_variable(name="b_ex_"+str(task_id), shape=[self.k_ex], scope="l%d"%layer)
                    
                    w_fc2_dummy = tf.concat([w_fc2, dummy_w],0)
                    
                    w_expanded = tf.concat([w_fc2_dummy, w_expand], 1)
                    b_expanded = tf.concat([b_fc2, b_expand], 0)
                    
                    self.params[w_fc2.name] = w_expanded
                    self.params[b_fc2.name] = b_expanded
                    self.h_fc2 = tf.nn.relu(tf.matmul(h_fc1, w_expanded) + b_expanded)
                    
        else:

            #flattened first fc layer
            W_fc1 = self.get_variable(name="w", scope="l1")   # layer-1 outgoing weight matrix
            b_fc1 = self.get_variable(name="b", scope="l1")

            h_fc1 = tf.nn.relu(tf.matmul(self.x, W_fc1) + b_fc1)

            #second fc layer
            W_fc2 = self.get_variable(name="w", scope="l2")   # layer-2 outgoing weight matrix
            b_fc2 = self.get_variable(name="b", scope="l2")

            self.h_fc2 = tf.nn.relu(tf.matmul(h_fc1, W_fc2) + b_fc2)

            self.w_fc = self.create_variable(name="w_"+str(task_id), shape=[self.h_fc2.shape[1], output_len], scope="l3", trainable=True)     # layer-3 outgoing weight matrix 
            self.b_fc = self.create_variable(name="b_"+str(task_id), shape=[output_len], scope="l3", trainable=True)
            y_conv = tf.matmul(self.h_fc2, self.w_fc) + self.b_fc

#         y_conv = tf.nn.sigmoid(y_conv)
        return y_conv
    
    def perform_selection(self, task_id, values_dict):      # Breadth first search for selecting non-zero units
        
        all_indices = defaultdict(list)     # to store indices of nonzero units
        selected_params = dict()            # to store values of nonzero units
        selected_prev_params = dict()
        
#         for scope, value in values_dict.items():    # Storing conv layers in selected parameters 
#             if "conv" in scope:
#                 selected_params[scope] = value
            
        for i in reversed(range(1,self.den_layers+1)):
            if i == self.den_layers:
                w = values_dict['l%d/w:0' % i]
                b = values_dict['l%d/b:0' % i]
                w_t = values_dict['l%d/w_%d:0' %(i,task_id)]
                b_t = values_dict['l%d/b_%d:0' %(i,task_id)]
                w = np.concatenate((w, w_t), 1)
                b = np.concatenate((b, b_t), 0)
                for j in range(w.shape[0]):
                    if w[j,-1] != 0.0:
                        all_indices['l%d' % i].append(j)
                # np.ix_(): fancy indexing, index with arrays of integers
                # Select non-zero weights between the last hidden layer and the output layer
                selected_params['l%d/w_%d:0' % (i, task_id)] = \
                    w[np.ix_(all_indices['l%d' % i], list(range(w.shape[1])))]
                selected_params['l%d/b_%d:0' % (i, task_id)] = b
            else:
                w = values_dict['l%d/w:0' % i]
                b = values_dict['l%d/b:0' % i]
                top_indices = all_indices['l%d' % (i + 1)]
                print("Layer %d: Selecting %d neurons" %(i+1, len(top_indices)))
                for j in range(w.shape[0]):
                    if np.count_nonzero(w[j, top_indices]) != 0 or i == 1:
                        all_indices['l%d' % i].append(j)
                
                # non-zero weights between the layer i and the layer i+1
                sub_weight = w[np.ix_(all_indices['l%d' % i], top_indices)]
                sub_biases = b[all_indices['l%d' % (i + 1)]]
                selected_params['l%d/w:0' % i] = sub_weight
                selected_params['l%d/b:0' % i] = sub_biases
                
                # prev_W : to avoid drastic change in value of weights (Regularization)
                selected_prev_params['l%d/w:0' % i] = \
                    self.prev_W['l%d/w:0' % i][np.ix_(all_indices['l%d' % i], top_indices)]
                selected_prev_params['l%d/b:0' % i] = \
                    self.prev_W['l%d/b:0' % i][all_indices['l%d' % (i + 1)]]

#         for keys, value in selected_params.items():
#             print(keys)
#             print(value)
                
        return [selected_params, selected_prev_params, all_indices]
        
    def build_SR(self, task_id, selected, output_len):    # creating selective retraining model
        
        self.initialize_parameters(output_len)
        h = self.x
        
        for i in range(1, self.den_layers):
            with tf.variable_scope('l%d' % i):
                w = tf.get_variable('w', initializer=selected['l%d/w:0' % i], trainable=True)
                b = tf.get_variable('b', initializer=selected['l%d/b:0' % i], trainable=True)
            h = tf.nn.relu(tf.matmul(h, w) + b)
            
        # last layer
        with tf.variable_scope('l%d' % self.den_layers):
            w = tf.get_variable('w_%d' % task_id,
                                initializer=selected['l%d/w_%d:0' % (self.den_layers, task_id)], trainable=True)
            b = tf.get_variable('b_%d' % task_id,
                                initializer=selected['l%d/b_%d:0' % (self.den_layers, task_id)], trainable=True)

        y_conv = tf.matmul(h, w) + b
#         y_conv = tf.nn.sigmoid(y_conv)
        return y_conv

    def optimization(self, prev_W=None):

        l2_regular = 0
        train_var = []
        regular_terms = []
        
        self.loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=self.y_, logits=y_conv))

#         for var in tf.trainable_variables():
#             l2_regular = l2_regular + tf.nn.l1_loss(var)
#             train_var.append(var)
#         print(len(train_var))

        l1_var = [var for var in tf.trainable_variables()]
        regularizer = tf.contrib.layers.l1_regularizer(self.l2_mu)
        reg_term = tf.contrib.layers.apply_regularization(regularizer, l1_var)
        self.loss = self.loss + reg_term
#         self.loss = self.loss + tf.reduce_mean(self.l2_mu * l2_regular)

        if prev_W != None:
            for var in l1_var:
                if var.name in prev_W.keys():
                    prev_w = prev_W[var.name]
                    regular_terms.append(tf.nn.l2_loss(var - prev_w))
            self.loss = self.loss + self.lamba_regular * tf.reduce_mean(regular_terms)
        
        opt = tf.train.AdamOptimizer(self.lr)
        grads = opt.compute_gradients(self.loss, l1_var)
        apply_grads = opt.apply_gradients(grads)

        correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(self.y_,1))
        self.acc_train = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
#         l1_var = [var for var in tf.trainable_variables()]
        l1_op_list = []
        with tf.control_dependencies([apply_grads]):  # exec apply_grads first
            for var in tf.trainable_variables():
                th_t = tf.fill(tf.shape(var), tf.convert_to_tensor(self.l1_thr))
                zero_t = tf.zeros(tf.shape(var))
#                 var_temp = var - (th_t * tf.sign(var))
                # [pseudo]:  if |var| < th_t: var = [0];  else: var = var_temp
                l1_op = var.assign(tf.where(tf.less(tf.abs(var), th_t), zero_t, var))
                l1_op_list.append(l1_op)
        
        with tf.control_dependencies(l1_op_list):
            self.train_model = tf.no_op()

    def train_task(self, task_id, batch_size, epochs, retraining=False, expansion=False):

        if task_id == 1:
            task_train_labels = tf.one_hot(indices=np.array(self.train_labels), depth=self.last_label_index+1)
        elif retraining == True or expansion == True:
            task_train_labels = tf.one_hot(indices=np.array(self.train_ex_labels), depth=self.last_label_index+1)
        else:
            task_train_labels = np.array(self.train_labels)
            task_train_labels = task_train_labels.reshape((task_train_labels.shape[0],1))
            task_train_labels = tf.convert_to_tensor(task_train_labels)

        dataset = tf.data.Dataset.from_tensor_slices((np.array(self.train), task_train_labels))
        dataset = dataset.shuffle(len(self.train_ex_labels)).repeat().batch(batch_size)
        iterator = dataset.make_one_shot_iterator()
        (x_data , y_data) = iterator.get_next()

        self.sess.run(tf.global_variables_initializer())
        task_train_labels = self.sess.run(task_train_labels)
        count = 0

#         for scope, ref in self.params.items():
#             if "l2" in scope:
#                 print(self.sess.run(ref))

        for i in range(epochs):

            for j in range(int(len(self.train)/batch_size)):
                x_batch , y_batch = self.sess.run([x_data,y_data])
                _, loss = self.sess.run([self.train_model, self.loss], feed_dict={self.x: x_batch, self.y_: y_batch})

            train_accuracy = self.acc_train.eval(session=self.sess, feed_dict={self.x: self.train, self.y_: task_train_labels})
            print("Epoch %d, training accuracy %g"%(i+1, train_accuracy))

            if train_accuracy == 1:
                count += 1

                if count > 9:
                    print("Best accuracy achieved! \n")
                    break
        
#         for scope, ref in self.params.items():
#             if "l2" in scope:
#                 print(self.sess.run(ref))
        
        return loss
    
    def predict(self, task_id, output_len=2):

        self.initialize_parameters(output_len)

        task_test_labels = tf.one_hot(indices=self.test_labels, depth=self.last_label_index+1)
        task_test_labels = self.sess.run(task_test_labels)
        
        #flattened first fc layer
        W_fc1 = self.get_variable(name="w", scope="l1")   # layer-1 outgoing weight matrix
        b_fc1 = self.get_variable(name="b", scope="l1")

        h_fc1 = tf.nn.relu(tf.matmul(self.x, W_fc1) + b_fc1)

        #second fc layer
        W_fc2 = self.get_variable(name="w", scope="l2")   # layer-2 outgoing weight matrix
        b_fc2 = self.get_variable(name="b", scope="l2")

        self.h_fc2 = tf.nn.relu(tf.matmul(h_fc1, W_fc2) + b_fc2)

        self.w_fc = self.get_variable(name="w", scope="l3")
        self.b_fc = self.get_variable(name="b", scope="l3")

        y_final = tf.matmul(self.h_fc2, self.w_fc) + self.b_fc
        y_final = tf.nn.softmax(y_final)
        
        self.sess.run(tf.global_variables_initializer())

        correct_prediction = tf.equal(tf.argmax(y_final,1), tf.argmax(self.y_,1))
        self.acc_test = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        test_accuracy = self.acc_test.eval(session=self.sess, feed_dict={self.x: self.test, self.y_: task_test_labels})
        return test_accuracy


In [5]:
if __name__ == "__main__":

    den = DEN()

    train_data, train_labels, test_data, test_labels, label_names = den.extract_data()
    np.random.shuffle(label_names)

# show image using cv
    # print(train_labels[512])
    # cv.imshow("", train_data[512])
    # cv.waitKey(0)

    tf.reset_default_graph()
    task_id = 0
    y_conv = None

    while den.last_label_index != (len(label_names) - 1):     # Loop for adding new tasks (lifelong learning)

        task_id += 1
        den.add_task(task_id, label_names)
        param_values = dict()
        selected = dict()
        print("-------------Training new task: %d--------------"%task_id)
        if task_id == 1:
            y_conv = den.build_model(task_id=task_id)
            den.optimization()
            _ = den.train_task(task_id=task_id, batch_size=den.batch_size, epochs=den.epochs)
            params = den.get_params()

            den.destroy_graph()
            den.restore_params(trainable=False, param_values=params)  # Freezes all learned weights making them non-trainable

            # ---------Performance-----------
            test_accuracy = den.predict(task_id=task_id, output_len=den.last_label_index+1)
            
        else:
            print("-----------Started Selective Retraining-------------")
            
            #------------------Selection-------------------
            print("---- Selecting nodes ----")
            y_conv = den.build_model(task_id=task_id, output_len=1)
            den.sess.run(tf.global_variables_initializer())
            params = den.get_params()
            den.optimization()
            
            _ = den.train_task(task_id=task_id, batch_size=den.batch_size, epochs=den.early_stop)
            params = den.get_params()
       
            [selected, selected_prev, all_indices] = den.perform_selection(task_id=task_id, values_dict=params)
            den.destroy_graph()

            #------------------Retraining-------------------
            print("\n---- Retraining selected nodes ----")
            y_conv = den.build_SR(task_id=task_id, selected=selected, output_len=den.last_label_index+1) 
            den.optimization(prev_W=selected_prev) 
            loss = den.train_task(task_id=task_id, batch_size=den.batch_size, epochs=den.epochs, retraining=True)

            print("Loss: %f"%loss)

            #--------------Performing Union----------------
            _vars = [(var.name, den.sess.run(var)) for var in tf.trainable_variables() if 'l' in var.name]

            for item in _vars:
                key, values = item
                selected[key] = values

            for i in reversed(range(1, den.den_layers+1)):
                if i == den.den_layers:
                    temp_weight = np.concatenate((params['l%d/w:0' % i], params['l%d/w_%d:0' % (i, task_id)]), axis=1)
                    temp_weight[np.ix_(all_indices['l%d' % i], list(range(den.last_label_index+1)))] = \
                        selected['l%d/w_%d:0' % (i, task_id)]
                    # Updating output matrix structure
                    params['l%d/w:0' % (i)] = temp_weight.tolist()
                    params['l%d/b:0' % (i)] = \
                        selected['l%d/b_%d:0' % (i, task_id)]

#                         params['l%d/w:0' % (i)] = np.concatenate([params['l%d/w:0' % (i)],params['l%d/w_%d:0' % (i,task_id)]], axis=1).tolist()
#                         params['l%d/b:0' % (i)] = np.concatenate([params['l%d/b:0' % (i)],params['l%d/b_%d:0' % (i,task_id)]], axis=0).tolist()
                else:
                    temp_weight = params['l%d/w:0' % i]
                    temp_biases = params['l%d/b:0' % i]
                    temp_weight[np.ix_(all_indices['l%d' % i], all_indices['l%d' % (i + 1)])] = \
                        selected['l%d/w:0' % i]
                    temp_biases[all_indices['l%d' % (i + 1)]] = \
                        selected['l%d/b:0' % i]
                    params['l%d/w:0' % i] = temp_weight
                    params['l%d/b:0' % i] = temp_biases

            den.destroy_graph()
            den.restore_params(trainable=False, param_values=params)  # Freezes all learned weights making them non-trainable

            # ---------Performance-----------
            test_accuracy = den.predict(task_id=task_id, output_len=den.last_label_index+1)
            
            if test_accuracy < 0.85:
                
                print("Overall accuracy: %g, lesser than threshold" %test_accuracy)
                print("-----------Started Dynamic Expansion------------")
                y_conv = den.build_model(task_id=task_id, expansion=True, output_len=den.last_label_index+1)
                den.optimization()
                _ = den.train_task(task_id=task_id, batch_size=den.batch_size, epochs=den.epochs, expansion=True)
                params = den.get_params()
                
                # deleting useless neurons added during expansion
                for i in reversed(range(1, den.den_layers)):
                    useless = []
                    prev_layer_weight = params['l%d/w:0'%i]
                    prev_layer_biases = params['l%d/b:0'%i]
                    
                    for j in range(prev_layer_weight.shape[1] - den.k_ex, prev_layer_weight.shape[1]):
                        if np.count_nonzero(prev_layer_weight[:, j]) == 0:
                            useless.append(j)
                    print(len(useless))

                    params['l%d/w:0'%i] = np.delete(prev_layer_weight, useless, axis = 1)
                    params['l%d/b:0'%i] = np.delete(prev_layer_biases, useless)
                    
                    next_layer_weight = params['l%d/w:0'%(i+1)]
                    params['l%d/w:0'%(i+1)] = np.delete(next_layer_weight, useless, axis = 0)
                    
                den.destroy_graph()
                den.restore_params(trainable=False, param_values=params)  # Freezes all learned weights making them non-trainable
                
                test_accuracy = den.predict(task_id=task_id, output_len=den.last_label_index+1)
        
        l2_n = den.get_variable(name="w", scope="l2")
        s = l2_n.get_shape().as_list()
        print(s)
        print("Final accuracy: %g \n" %test_accuracy)     
    


..........loading dataset from numpy files..........

 
 Added new category: night_stand
 
 Added new category: bench
Train size: 373
Test size: 106
-------------Training new task: 1--------------
Epoch 1, training accuracy 0.989276
Epoch 2, training accuracy 1
Epoch 3, training accuracy 0.994638
Epoch 4, training accuracy 1
Epoch 5, training accuracy 1
Epoch 6, training accuracy 0.994638
Epoch 7, training accuracy 1
Epoch 8, training accuracy 1
Epoch 9, training accuracy 1
Epoch 10, training accuracy 0.997319
Epoch 11, training accuracy 1
Epoch 12, training accuracy 0.994638
Epoch 13, training accuracy 0.997319
Epoch 14, training accuracy 1
Epoch 15, training accuracy 1
Epoch 16, training accuracy 1
Best accuracy achieved! 

[1024, 256]
Final accuracy: 0.990566 

 
 Added new category: bed

 Less samples; so taking them all

 Less samples; so taking them all
Train size: 888
Test size: 206
-------------Training new task: 2--------------
-----------Started Selective Retraining---------

Epoch 39, training accuracy 0.994975
Epoch 40, training accuracy 0.969849
Epoch 41, training accuracy 0.994975
Epoch 42, training accuracy 0.994975
Epoch 43, training accuracy 0.994975
Epoch 44, training accuracy 0.98995
Epoch 45, training accuracy 1
Epoch 46, training accuracy 0.997487
Epoch 47, training accuracy 1
Epoch 48, training accuracy 1
Epoch 49, training accuracy 0.997487
Epoch 50, training accuracy 0.997487
Loss: 0.483854
[1024, 256]
Final accuracy: 0.869318 

 
 Added new category: chair

 Less samples; so taking them all

 Less samples; so taking them all
Train size: 1751
Test size: 452
-------------Training new task: 7--------------
-----------Started Selective Retraining-------------
---- Selecting nodes ----
Epoch 1, training accuracy 1
Epoch 2, training accuracy 1
Epoch 3, training accuracy 1
Epoch 4, training accuracy 1
Epoch 5, training accuracy 1
Layer 3: Selecting 250 neurons
Layer 2: Selecting 1024 neurons

---- Retraining selected nodes ----
Epoch 1, training acc

Epoch 26, training accuracy 0.998102
Epoch 27, training accuracy 0.998102
Epoch 28, training accuracy 0.998102
Epoch 29, training accuracy 0.998102
Epoch 30, training accuracy 0.998102
Epoch 31, training accuracy 0.998102
Epoch 32, training accuracy 0.998102
Epoch 33, training accuracy 0.996205
Epoch 34, training accuracy 0.998102
Epoch 35, training accuracy 0.998102
Epoch 36, training accuracy 0.998102
Epoch 37, training accuracy 0.998102
Epoch 38, training accuracy 0.998102
Epoch 39, training accuracy 0.998102
Epoch 40, training accuracy 0.996205
Epoch 41, training accuracy 0.998102
Epoch 42, training accuracy 0.996205
Epoch 43, training accuracy 0.998102
Epoch 44, training accuracy 0.998102
Epoch 45, training accuracy 0.998102
Epoch 46, training accuracy 0.998102
Epoch 47, training accuracy 0.998102
Epoch 48, training accuracy 0.996205
Epoch 49, training accuracy 0.998102
Epoch 50, training accuracy 0.996205
0
0
[1029, 261]
Final accuracy: 0.845238 

 
 Added new category: bottle
Tr

Epoch 32, training accuracy 0.986888
Epoch 33, training accuracy 0.987762
Epoch 34, training accuracy 0.987762
Epoch 35, training accuracy 0.986888
Epoch 36, training accuracy 0.98951
Epoch 37, training accuracy 0.987762
Epoch 38, training accuracy 0.98951
Epoch 39, training accuracy 0.987762
Epoch 40, training accuracy 0.986888
Epoch 41, training accuracy 0.986014
Epoch 42, training accuracy 0.987762
Epoch 43, training accuracy 0.987762
Epoch 44, training accuracy 0.986014
Epoch 45, training accuracy 0.987762
Epoch 46, training accuracy 0.987762
Epoch 47, training accuracy 0.986888
Epoch 48, training accuracy 0.986888
Epoch 49, training accuracy 0.98951
Epoch 50, training accuracy 0.986014
0
0
[1034, 266]
Final accuracy: 0.856502 

 
 Added new category: plant
Train size: 520
Test size: 992
-------------Training new task: 14--------------
-----------Started Selective Retraining-------------
---- Selecting nodes ----
Epoch 1, training accuracy 1
Epoch 2, training accuracy 1
Epoch 3, tr

Epoch 42, training accuracy 0.988739
Epoch 43, training accuracy 0.997748
Epoch 44, training accuracy 0.995495
Epoch 45, training accuracy 0.997748
Epoch 46, training accuracy 0.993243
Epoch 47, training accuracy 0.997748
Epoch 48, training accuracy 1
Epoch 49, training accuracy 0.995495
Epoch 50, training accuracy 0.993243
Loss: 0.671989
Overall accuracy: 0.834244, lesser than threshold
-----------Started Dynamic Expansion------------
Epoch 1, training accuracy 0.993243
Epoch 2, training accuracy 0.993243
Epoch 3, training accuracy 0.993243
Epoch 4, training accuracy 0.993243
Epoch 5, training accuracy 0.993243
Epoch 6, training accuracy 0.993243
Epoch 7, training accuracy 0.993243
Epoch 8, training accuracy 0.993243
Epoch 9, training accuracy 0.993243
Epoch 10, training accuracy 0.993243
Epoch 11, training accuracy 0.993243
Epoch 12, training accuracy 0.993243
Epoch 13, training accuracy 0.993243
Epoch 14, training accuracy 0.993243
Epoch 15, training accuracy 0.993243
Epoch 16, trai

...

Epoch 36, training accuracy 0.944403
Epoch 37, training accuracy 0.97145
Epoch 38, training accuracy 0.957926
Epoch 39, training accuracy 0.956424
Epoch 40, training accuracy 0.951916
Epoch 41, training accuracy 0.956424
Epoch 42, training accuracy 0.978963
Epoch 43, training accuracy 0.936138
Epoch 44, training accuracy 0.961683
Epoch 45, training accuracy 0.969947
Epoch 46, training accuracy 0.952667
Epoch 47, training accuracy 0.958678
Epoch 48, training accuracy 0.97145
Epoch 49, training accuracy 0.966942
Epoch 50, training accuracy 0.951916
Loss: 0.915628
Overall accuracy: 0.768641, lesser than threshold
-----------Started Dynamic Expansion------------
Epoch 1, training accuracy 0.961683
Epoch 2, training accuracy 0.967693
Epoch 3, training accuracy 0.974455
Epoch 4, training accuracy 0.978963
Epoch 5, training accuracy 0.983471
Epoch 6, training accuracy 0.987979
Epoch 7, training accuracy 0.987979
Epoch 8, training accuracy 0.987979
Epoch 9, training accuracy 0.98873
Epoch 10, 

Epoch 27, training accuracy 0.949086
Epoch 28, training accuracy 0.983029
Epoch 29, training accuracy 0.946475
Epoch 30, training accuracy 0.983029
Epoch 31, training accuracy 0.97389
Epoch 32, training accuracy 0.981723
Epoch 33, training accuracy 0.977807
Epoch 34, training accuracy 0.964752
Epoch 35, training accuracy 0.988251
Epoch 36, training accuracy 0.968668
Epoch 37, training accuracy 0.95953
Epoch 38, training accuracy 0.969974
Epoch 39, training accuracy 0.976501
Epoch 40, training accuracy 0.964752
Epoch 41, training accuracy 0.977807
Epoch 42, training accuracy 0.960836
Epoch 43, training accuracy 0.964752
Epoch 44, training accuracy 0.975196
Epoch 45, training accuracy 0.980418
Epoch 46, training accuracy 0.963446
Epoch 47, training accuracy 0.963446
Epoch 48, training accuracy 0.97389
Epoch 49, training accuracy 0.960836
Epoch 50, training accuracy 0.976501
Loss: 1.262582
Overall accuracy: 0.773719, lesser than threshold
-----------Started Dynamic Expansion------------
E

Epoch 18, training accuracy 0.985879
Epoch 19, training accuracy 0.96534
Epoch 20, training accuracy 0.97561
Epoch 21, training accuracy 0.983312
Epoch 22, training accuracy 0.967908
Epoch 23, training accuracy 0.96534
Epoch 24, training accuracy 0.970475
Epoch 25, training accuracy 0.971759
Epoch 26, training accuracy 0.982028
Epoch 27, training accuracy 0.984596
Epoch 28, training accuracy 0.971759
Epoch 29, training accuracy 0.948652
Epoch 30, training accuracy 0.97561
Epoch 31, training accuracy 0.982028
Epoch 32, training accuracy 0.97561
Epoch 33, training accuracy 0.970475
Epoch 34, training accuracy 0.987163
Epoch 35, training accuracy 0.958922
Epoch 36, training accuracy 0.982028
Epoch 37, training accuracy 0.970475
Epoch 38, training accuracy 0.980745
Epoch 39, training accuracy 0.967908
Epoch 40, training accuracy 0.962773
Epoch 41, training accuracy 0.96534
Epoch 42, training accuracy 0.955071
Epoch 43, training accuracy 0.955071
Epoch 44, training accuracy 0.988447
Epoch 4

Epoch 10, training accuracy 0.909311
Epoch 11, training accuracy 0.944377
Epoch 12, training accuracy 0.970979
Epoch 13, training accuracy 0.958888
Epoch 14, training accuracy 0.962515
Epoch 15, training accuracy 0.954051
Epoch 16, training accuracy 0.951632
Epoch 17, training accuracy 0.970979
Epoch 18, training accuracy 0.937122
Epoch 19, training accuracy 0.935913
Epoch 20, training accuracy 0.960097
Epoch 21, training accuracy 0.943168
Epoch 22, training accuracy 0.946796
Epoch 23, training accuracy 0.946796
Epoch 24, training accuracy 0.932285
Epoch 25, training accuracy 0.957678
Epoch 26, training accuracy 0.943168
Epoch 27, training accuracy 0.95526
Epoch 28, training accuracy 0.963724
Epoch 29, training accuracy 0.902056
Epoch 30, training accuracy 0.943168
Epoch 31, training accuracy 0.958888
Epoch 32, training accuracy 0.944377
Epoch 33, training accuracy 0.977025
Epoch 34, training accuracy 0.94075
Epoch 35, training accuracy 0.962515
Epoch 36, training accuracy 0.958888
Epo

Epoch 1, training accuracy 0.878715
Epoch 2, training accuracy 0.911647
Epoch 3, training accuracy 0.921285
Epoch 4, training accuracy 0.944578
Epoch 5, training accuracy 0.944578
Epoch 6, training accuracy 0.969478
Epoch 7, training accuracy 0.926104
Epoch 8, training accuracy 0.934137
Epoch 9, training accuracy 0.946185
Epoch 10, training accuracy 0.953414
Epoch 11, training accuracy 0.953414
Epoch 12, training accuracy 0.933333
Epoch 13, training accuracy 0.949398
Epoch 14, training accuracy 0.95502
Epoch 15, training accuracy 0.959036
Epoch 16, training accuracy 0.945382
Epoch 17, training accuracy 0.935743
Epoch 18, training accuracy 0.959036
Epoch 19, training accuracy 0.964659
Epoch 20, training accuracy 0.95743
Epoch 21, training accuracy 0.922892
Epoch 22, training accuracy 0.963052
Epoch 23, training accuracy 0.944578
Epoch 24, training accuracy 0.947791
Epoch 25, training accuracy 0.941365
Epoch 26, training accuracy 0.956627
Epoch 27, training accuracy 0.946988
Epoch 28, tr